In [47]:
%reset -sf

In [48]:
#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# available on Google, AtCoder Python3, not available on Codeforces
# import numpy as np
# import scipy

M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
# d8 = [(1,0),(1,1),(0,1),(-1,1),(-1,0),(-1,-1),(0,-1),(1,-1)]
# d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [49]:
# trr = ""
# srr = """
# """.strip()


# trr = ""
with open("/Users/hkmac/Downloads/input.txt") as f:
    srr_inp = f.read().strip()

with open("input1.txt") as f:
    srr_inp = f.read().strip()
    
srr_inp = list(srr_inp.split("\n\n"))
srr_inp = [srr_in.strip().split("\n")[1:] for srr_in in srr_inp]
srr_inp = [[list(map(int,srr_i.split(","))) for srr_i in srr_in] for srr_in in srr_inp]

In [50]:
len(srr_inp)

5

In [59]:
import copy

mapping = {}

for ii in range(len(srr_inp)):
    for jj in range(len(srr_inp)):
        if ii == jj:
            continue
        
        arr = srr_inp[ii]
        brr = srr_inp[jj]

        for px,py,pz in itertools.product([-1,1], repeat=3):
            for transform in [
    lambda x: (x[0],x[1],x[2]),
    lambda x: (x[0],x[2],x[1]),
    lambda x: (x[1],x[0],x[2]),
    lambda x: (x[1],x[2],x[0]),
    lambda x: (x[2],x[0],x[1]),
    lambda x: (x[2],x[1],x[0]),
]:
                xrr = [transform((x*px,y*py,z*pz)) for x,y,z in arr]

                for (p,q,r) in (xrr):
                    for (p2,q2,r2) in (brr):
                        p9,q9,r9 = p-p2, q-q2, r-r2
                        
                        tf = lambda x: (transform((x[0]*px,x[1]*py,x[2]*pz),)[0] - p9, 
                                        transform((x[0]*px,x[1]*py,x[2]*pz),)[1] - q9, 
                                        transform((x[0]*px,x[1]*py,x[2]*pz),)[2] - r9)
                        
                        prr = [tf((x,y,z),) for x,y,z in arr]
                        qrr = [(x,y,z) for x,y,z in brr]

                        if len(set(prr) & set(qrr)) >= 12:
                            mapping[ii,jj] = (transform, px, py, pz, p9, q9, r9)
                            print(ii,jj,"ok")
                            print(id(tf))
                            print(tf((1,2,3),))
                            break
                    if (ii,jj) in mapping:
                        break
                if (ii,jj) in mapping:
                    break
            if (ii,jj) in mapping:
                break

0 1 ok
140316999422448
(67, 1248, -46)
1 0 ok
140316999422448
(67, -1244, -46)
1 3 ok
140316996781968
(-159, 1136, 26)
1 4 ok
140316996783120
(-1107, -87, 111)
2 4 ok
140316996781392
(170, -1124, 69)
3 1 ok
140316996783552
(161, -1132, -20)
4 1 ok
140316996781968
(90, 110, -1105)
4 2 ok
140316996781824
(1127, -167, 69)


In [60]:
for k,func in mapping.items():
    print(k, func)
#     print(func((1,2,3),))

(0, 1) (<function <lambda> at 0x7f9e18b510d0>, -1, 1, -1, -68, -1246, 43)
(1, 0) (<function <lambda> at 0x7f9e18e81310>, -1, 1, -1, -68, 1246, 43)
(1, 3) (<function <lambda> at 0x7f9e18e819d0>, 1, 1, 1, 160, -1134, -23)
(1, 4) (<function <lambda> at 0x7f9e18b23f70>, 1, -1, -1, 1104, 88, -113)
(2, 4) (<function <lambda> at 0x7f9e18b0eca0>, 1, 1, -1, -168, 1125, -72)
(3, 1) (<function <lambda> at 0x7f9e18b0e9d0>, 1, 1, 1, -160, 1134, 23)
(4, 1) (<function <lambda> at 0x7f9e1894ae50>, -1, 1, -1, -88, -113, 1104)
(4, 2) (<function <lambda> at 0x7f9e18e811f0>, 1, 1, -1, -1125, 168, -72)


In [53]:
stack = [0]
visited = set(stack)
prev = {}

while stack:
    cur = stack.pop()
    for nex in range(len(srr_inp)):
        if nex in visited:
            continue
        if (cur,nex) not in mapping:
            continue
        stack.append(nex)
        visited.add(nex)
        prev[nex] = cur

In [54]:
visited

{0, 1, 2, 3, 4}

In [55]:
prev

{1: 0, 3: 1, 4: 1, 2: 4}

In [ ]:
mapping[4,1](1,2,3)

In [61]:
beacons = set(map(tuple, srr_inp[0]))

for i in range(len(srr_inp)):
    cur = i
    arr = srr_inp[i]
    transforms = []
    while cur != 0:
        transform, px, py, pz, p9, q9, r9 = mapping[cur, prev[cur]]

        tf = lambda x: (transform((x[0]*px,x[1]*py,x[2]*pz),)[0] - p9, 
                        transform((x[0]*px,x[1]*py,x[2]*pz),)[1] - q9, 
                        transform((x[0]*px,x[1]*py,x[2]*pz),)[2] - r9)
        arr = [tf((x,y,z),) for x,y,z in arr]        
        cur = prev[cur]
    assert cur == 0

    beacons.update(set(map(tuple, arr)))

In [62]:
beacons

{(-892, 524, 684),
 (-876, 649, 763),
 (-838, 591, 734),
 (-789, 900, -551),
 (-739, -1745, 668),
 (-706, -3180, -659),
 (-697, -3072, -689),
 (-689, 845, -530),
 (-687, -1600, 576),
 (-661, -816, -575),
 (-654, -3158, -753),
 (-635, -1737, 486),
 (-631, -672, 1502),
 (-624, -1620, 1868),
 (-620, -3212, 371),
 (-618, -824, -621),
 (-612, -1695, 1788),
 (-601, -1648, -643),
 (-584, 868, -557),
 (-537, -823, -458),
 (-532, -1715, 1894),
 (-518, -1681, -600),
 (-499, -1607, -770),
 (-485, -357, 347),
 (-470, -3283, 303),
 (-456, -621, 1527),
 (-447, -329, 318),
 (-430, -3130, 366),
 (-413, -627, 1469),
 (-345, -311, 381),
 (-36, -1284, 1171),
 (-27, -1108, -65),
 (7, -33, -71),
 (12, -2351, -103),
 (26, -1119, 1091),
 (346, -2985, 342),
 (366, -3059, 397),
 (377, -2827, 367),
 (390, -675, -793),
 (396, -1931, -563),
 (404, -588, -901),
 (408, -1815, 803),
 (423, -701, 434),
 (432, -2009, 850),
 (443, 580, 662),
 (455, 729, 728),
 (456, -540, 1869),
 (459, -707, 401),
 (465, -695, 1988),
 

In [63]:
len(beacons)

79